In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib.pyplot as plt
import seaborn as sns

/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Daily_Ridership_Data__Beginning_2020.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload


In [92]:
monthly_riders_since_2008 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv')

#No missing values
monthly_riders_since_2008.dropna(inplace=True)

#Filter to only include Subway and no other MTA agency
monthly_riders_since_2008 = monthly_riders_since_2008[monthly_riders_since_2008['Agency'] == 'Subway'] 

#Rename "Month" column into Date
monthly_riders_since_2008.rename(columns = {'Month':'Date'}, inplace = True)

#Polish Date column and conver to datetime
monthly_riders_since_2008['Date'] = monthly_riders_since_2008['Date'].str.replace('/01/', '/', regex=True)
monthly_riders_since_2008['Date'] = pd.to_datetime(monthly_riders_since_2008['Date'], format='%m/%Y')
#monthly_riders_since_2008['Date'] = monthly_riders_since_2008['Date'].dt.strftime('%m/%Y')

#Break out date into coumns for date, month, and year
monthly_riders_since_2008['Month'] = monthly_riders_since_2008['Date'].dt.month
monthly_riders_since_2008['Year'] = monthly_riders_since_2008['Date'].dt.year

#Drop Date column now
monthly_riders_since_2008.drop(['Date', 'Agency'], inplace=True, axis=1)


#Splitting Month column into Year and Month values
#monthly_riders_since_2008[['Month', 'Year']] = monthly_riders_since_2008['Date'].str.split('/', expand=True)

#Filter out the year 2024
monthly_riders_since_2008 = monthly_riders_since_2008[monthly_riders_since_2008['Year'] < 2024] 

#Groupby Year and look at mean ridership
monthly_riders_since_2008['Mean Ridership By Year'] = monthly_riders_since_2008.groupby('Year')['Ridership'].transform(lambda x: x.mean())
monthly_riders_since_2008['Total Ridership By Year'] = monthly_riders_since_2008.groupby('Year')['Ridership'].transform(lambda x: x.sum())


#Rearrange
monthly_riders_since_2008 = monthly_riders_since_2008.reindex(['Month', 'Year', 'Ridership', 'Mean Ridership By Year', 'Total Ridership By Year'], axis=1)


groued_by_year = monthly_riders_since_2008[['Year', 'Mean Ridership By Year', 'Total Ridership By Year']].drop_duplicates()
groued_by_year



#sns.barplot(data=by_year, x='Year', y='Mean Ridership By Year')
#plt.show()


print(monthly_riders_since_2008)
print(groued_by_year)

     Month  Year  Ridership  Mean Ridership By Year  Total Ridership By Year
238      1  2018  134683435            1.400050e+08               1680060402
242      2  2018  127432835            1.400050e+08               1680060402
246      3  2018  143982923            1.400050e+08               1680060402
251      4  2018  141950369            1.400050e+08               1680060402
255      5  2018  150320833            1.400050e+08               1680060402
..     ...   ...        ...                     ...                      ...
673      8  2023   94141961            9.582530e+07               1149903621
679      9  2023   95253595            9.582530e+07               1149903621
686     10  2023  103949018            9.582530e+07               1149903621
693     11  2023   98511566            9.582530e+07               1149903621
700     12  2023   95685578            9.582530e+07               1149903621

[72 rows x 5 columns]
     Year  Mean Ridership By Year  Total Ridership By

In [62]:
daily_riders_since_2020 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Daily_Ridership_Data__Beginning_2020.csv', parse_dates=['Date'])

#No missing values
daily_riders_since_2020.dropna(inplace=True)

#Drop unccesary columns that belong to other agencies outside of the subway 
daily_riders_since_2020.drop(daily_riders_since_2020.iloc[:, 3:], inplace=True, axis=1)

#Convert to Datetime
#daily_riders_since_2020['Date'] = pd.to_datetime(daily_riders_since_2020['Date'])

#Break out date into columns for date, month, and year
daily_riders_since_2020['Day'] = daily_riders_since_2020['Date'].dt.day
daily_riders_since_2020['Month'] = daily_riders_since_2020['Date'].dt.month
daily_riders_since_2020['Year'] = daily_riders_since_2020['Date'].dt.year

#Sort by date
daily_riders_since_2020.sort_values('Date', ascending=True, inplace=True)




daily_riders_since_2020

,Date,Subways: Total Estimated Ridership,Subways: % of Comparable Pre-Pandemic Day,Day,Month,Year
1542,2020-03-02,5329915,0.96,2,3,2020
1541,2020-03-03,5481103,0.98,3,3,2020
1540,2020-03-04,5498809,0.99,4,3,2020
1539,2020-03-05,5496453,0.99,5,3,2020
1538,2020-03-06,5189447,0.93,6,3,2020
...,...,...,...,...,...,...
4,2024-05-19,2121168,0.89,19,5,2024
3,2024-05-20,3677281,0.64,20,5,2024
2,2024-05-21,4024394,0.70,21,5,2024
1,2024-05-22,4096783,0.72,22,5,2024


In [93]:
#df3 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload', iterator=True, chunksize=1000)
hourly_ridership_since_2022 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload')


#dropping missing values
hourly_ridership_since_2022.dropna(inplace=True)

#Convert Station ID to int (theres a station id called 'TRAM1'? check if its true)
#hourly_ridership_since_2022['station_complex_id'] = hourly_ridership_since_2022['station_complex_id'].astype(int) 

#Convert timestamp column to Datetime
hourly_ridership_since_2022['transit_timestamp'] = pd.to_datetime(hourly_ridership_since_2022['transit_timestamp'], format='%m/%d/%Y %I:%M:%S %p')

#Breakout timestamp info into diffeent columns
hourly_ridership_since_2022['Day'] = hourly_ridership_since_2022['transit_timestamp'].dt.day
hourly_ridership_since_2022['Month'] = hourly_ridership_since_2022['transit_timestamp'].dt.month
hourly_ridership_since_2022['Year'] = hourly_ridership_since_2022['transit_timestamp'].dt.year

#Filter out Staten Island
hourly_ridership_since_2022 = hourly_ridership_since_2022[hourly_ridership_since_2022['borough'] != 'Staten Island'] 

#Drop unccesary columns for our analysis
hourly_ridership_since_2022.drop(columns=['payment_method', 'fare_class_category', 'transfers'], inplace=True, axis=1)


for column in hourly_ridership_since_2022.columns:
    print(column,':',pd.api.types.infer_dtype(hourly_ridership_since_2022[column]))

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.